In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# ✅ Ensure "static" directory exists for saving images
if not os.path.exists("static"):
    os.makedirs("static")

# 📌 **1. Load the dataset**
df = pd.read_csv("Groceries_dataset.csv")

# 📌 **2. Convert transactions into list format**
transactions = df.groupby("Member_number")["itemDescription"].apply(list).tolist()

# 📌 **3. Remove duplicate items within each transaction**
transactions = [list(set(t)) for t in transactions]

# 📌 **4. Convert transactions to one-hot encoded DataFrame**
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 📌 **5. Find frequent itemsets using Apriori**
frequent_itemsets = apriori(df_encoded, min_support=0.02, use_colnames=True)

# 📌 **6. Generate association rules**
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

# ✅ Convert frozensets to strings for visualization
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x)))
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x)))

# ✅ Ensure required columns exist for plotting
if "support" not in rules.columns:
    rules["support"] = 0.05  # Placeholder values if missing
if "lift" not in rules.columns:
    rules["lift"] = 1.5  # Placeholder values if missing

# 📌 **7. Save Heatmap (Top 50 Rules)**
top_rules = rules.nlargest(50, "support")
pivot_table = top_rules.pivot(index="antecedents", columns="consequents", values="support")

plt.figure(figsize=(18, 12))
sns.heatmap(pivot_table, cmap="YlGnBu", annot=True, fmt=".2f", linewidths=0.5)
plt.xticks(rotation=45, ha="right", fontsize=12)
plt.yticks(fontsize=12)
plt.title("Top 50 Association Rules Heatmap", fontsize=14)
plt.savefig("static/heatmap.png")  # ✅ Save as image
plt.close()

# 📌 **8. Save Scatter Plot (Support vs. Confidence)**
plt.figure(figsize=(10, 6))
plt.scatter(rules["support"], rules["confidence"], alpha=0.6, c=rules["lift"], cmap="viridis")
plt.colorbar(label="Lift")
plt.xlabel("Support")
plt.ylabel("Confidence")
plt.title("Support vs. Confidence (Color = Lift)")
plt.savefig("static/scatter_plot.png")  # ✅ Save as image
plt.close()

# 📌 **9. Save Network Graph of Association Rules**
import networkx as nx
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# ✅ Create a directed graph
G = nx.DiGraph()
for _, row in rules.iterrows():
    G.add_edge(row["antecedents"], row["consequents"], weight=row["confidence"])

# ✅ Extract confidence values
confidence_values = [d["weight"] for _, _, d in G.edges(data=True)]
cmap = cm.Blues
norm = mcolors.Normalize(vmin=min(confidence_values), vmax=max(confidence_values))

# ✅ Draw graph
fig, ax = plt.subplots(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_edges(G, pos, edge_color=confidence_values, edge_cmap=cmap, edge_vmin=min(confidence_values), edge_vmax=max(confidence_values), ax=ax)
nx.draw_networkx_nodes(G, pos, node_color="lightblue", edgecolors="black", linewidths=1.5, ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight="bold", ax=ax)

# ✅ Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label("Confidence", fontsize=12)

plt.title("Network Graph of Association Rules", fontsize=14)
plt.savefig("static/network_graph.png")  # ✅ Save as image
plt.close()

print("All visualizations saved successfully in 'static' directory!")


All visualizations saved successfully in 'static' directory!


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask, render_template

app = Flask(__name__)

# Load dataset
df = pd.read_csv("Groceries_dataset.csv")

# Convert date column to datetime if necessary
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Drop non-numeric columns for heatmap
df_numeric = df.select_dtypes(include=["number"])

# Generate Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_numeric.corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Heatmap of Dataset")
plt.savefig("static/heatmap.png")  # Save the image
plt.close()

# Generate Scatter Plot (modify based on your dataset)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_numeric.columns[0], y=df_numeric.columns[1], data=df_numeric)
plt.title("Scatter Plot")
plt.savefig("static/scatterplot.png")  # Save the image
plt.close()

@app.route("/")
def home():
    return render_template("index.html")

if __name__ == "__main__":
    app.run(debug=False)


In [2]:
if __name__ == "__main__":
    app.run(debug=False)



NameError: name 'app' is not defined